In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [2]:
df=pd.read_csv("insurance.csv")
df.head()


,age,affordibility,bought_insurance
0,22,1,0
1,25,0,0
2,47,1,1
3,52,0,0
4,46,1,1


In [3]:
X=df.drop(["bought_insurance"],axis=1)
y=df.iloc[:,2]

In [4]:
y

0     0
1     0
2     1
3     0
4     1
5     1
6     0
7     1
8     1
9     1
10    0
11    0
12    0
13    0
14    1
15    1
16    1
17    1
18    0
19    0
20    0
21    0
22    1
23    1
24    1
25    1
26    0
27    0
Name: bought_insurance, dtype: int64

In [5]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=2)


In [11]:
X_train_scaled=X_train.copy()
X_train_scaled['age']=X_train_scaled['age']/100
X_test_scaled=X_test.copy()
X_test_scaled['age']=X_test_scaled['age']/100


In [12]:
X_test_scaled.head()

,age,affordibility
1,0.25,0
0,0.22,1
14,0.49,1
9,0.61,1
19,0.18,1


In [8]:
model=keras.Sequential([
                        keras.layers.Dense(1,input_shape=(2,),activation='sigmoid',
                        kernel_initializer='one',bias_initializer='zeros')
])
model.compile(optimizer='adam',loss='binary_crossentropy',metrics='accuracy')
model.fit(X_train_scaled,y_train,epochs=6000)

Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 13ms/step - loss: 0.5119 - accuracy: 0.8182
Epoch 3502/6000
1/1 [==============================] - 0s 15ms/step - loss: 0.5119 - accuracy: 0.8182
Epoch 3503/6000
1/1 [==============================] - 0s 12ms/step - loss: 0.5119 - accuracy: 0.8182
Epoch 3504/6000
1/1 [==============================] - 0s 14ms/step - loss: 0.5118 - accuracy: 0.8182
Epoch 3505/6000
1/1 [==============================] - 0s 12ms/step - loss: 0.5118 - accuracy: 0.8182
Epoch 3506/6000
1/1 [==============================] - 0s 13ms/step - loss: 0.5118 - accuracy: 0.8182
Epoch 3507/6000
1/1 [==============================] - 0s 16ms/step - loss: 0.5117 - accuracy: 0.8182
Epoch 3508/6000
1/1 [==============================] - 0s 13ms/step - loss: 0.5117 - accuracy: 0.8182
Epoch 3509/6000
1/1 [==============================] - 0s 12ms/step - loss: 0.5117 - accuracy: 0.8182
Epoch 3510/6000
1/1 [==========================

In [26]:
model.evaluate(X_test_scaled,y_test)

1/1 [==============================] - 0s 27ms/step - loss: 0.2740 - accuracy: 1.0000


[0.27396056056022644, 1.0]

In [27]:
model.predict(X_test_scaled)

array([[0.13526127],
       [0.37258807],
       [0.7413501 ],
       [0.8522817 ],
       [0.31987762],
       [0.8288794 ]], dtype=float32)

In [15]:
y_test

1     0
0     0
14    1
9     1
19    0
17    1
Name: bought_insurance, dtype: int64

In [28]:
coef, intercept = model.get_weights()
coef, intercept


(array([[5.8301024],
        [1.5089923]], dtype=float32), array([-3.3127449], dtype=float32))

In [32]:
def sigmoid(x):
  import math
  return 1/(1+math.exp(-x))
sigmoid(5)

0.9933071490757153

In [55]:
def prediction_function(age,affordibility):
  weight_sum=coef[0]*age+coef[1]*affordibility+intercept
  return sigmoid(weight_sum)

In [56]:
prediction_function(.25,0)

0.13526126165800964

In [59]:
def sigmoid_numpy(X):
   return 1/(1+np.exp(-X))

sigmoid_numpy(np.array([12,0,1]))

array([0.99999386, 0.5       , 0.73105858])

In [60]:
def log_loss(y_true, y_predicted):
    epsilon = 1e-15
    y_predicted_new = [max(i,epsilon) for i in y_predicted]
    y_predicted_new = [min(i,1-epsilon) for i in y_predicted_new]
    y_predicted_new = np.array(y_predicted_new)
    return -np.mean(y_true*np.log(y_predicted_new)+(1-y_true)*np.log(1-y_predicted_new))

In [61]:
def gradient_descent(age, affordability, y_true, epochs, loss_thresold):
    w1 = w2 = 1
    bias = 0
    rate = 0.5
    n = len(age)
    for i in range(epochs):
        weighted_sum = w1 * age + w2 * affordability + bias
        y_predicted = sigmoid_numpy(weighted_sum)
        loss = log_loss(y_true, y_predicted)

        w1derivative = (1/n)*np.dot(np.transpose(age),(y_predicted-y_true)) 
        w2derivative = (1/n)*np.dot(np.transpose(affordability),(y_predicted-y_true)) 

        bias_derivative = np.mean(y_predicted-y_true)
        w1 = w1 - rate * w1derivative
        w2 = w2 - rate * w2derivative
        bias = bias - rate * bias_derivative

        print (f'Epoch:{i}, w1:{w1}, w2:{w2}, bias:{bias}, loss:{loss}')

        if loss<=loss_thresold:
            break

    return w1, w2, bias

In [62]:
gradient_descent(X_train_scaled['age'],X_train_scaled['affordibility'],y_train,1000, 0.4631)


Epoch:0, w1:0.973391914875179, w2:0.9484790550524685, bias:-0.11334365204477419, loss:0.714610156590683
Epoch:1, w1:0.9526050112606814, w2:0.9061888855737805, bias:-0.21202325373024938, loss:0.6844545754122794
Epoch:2, w1:0.9371389216394038, w2:0.8724863718378795, bias:-0.29735477734867394, loss:0.6623114062843336
Epoch:3, w1:0.9263972300850559, w2:0.8464964034855741, bias:-0.3708685783767046, loss:0.6463299135248588
Epoch:4, w1:0.9197530021886515, w2:0.8272363836850969, bias:-0.4341501414207024, loss:0.6349055537023248
Epoch:5, w1:0.9165978234998884, w2:0.8137152861544186, bias:-0.48872346395231214, loss:0.6267483980102484
Epoch:6, w1:0.9163723575035057, w2:0.804999539668341, bias:-0.5359802497218649, loss:0.6208785684684357
Epoch:7, w1:0.9185813941347364, w2:0.8002492021116546, bias:-0.5771466732016118, loss:0.6165824439197473
Epoch:8, w1:0.9227979840062284, w2:0.7987322635165252, bias:-0.6132761019356342, loss:0.6133562082414478
Epoch:9, w1:0.9286609028533963, w2:0.7998249643661494,

(5.776540098602374, 1.5072936283526548, -3.292466234362901)

In [63]:
coef,intercept

(array([[5.8301024],
        [1.5089923]], dtype=float32), array([-3.3127449], dtype=float32))